In [1]:
import torch
import pypose as pp

# 1. Intialization

In [2]:
x = pp.identity_SE3(2,1)
y = pp.randn_se3(2,2)
n = torch.randn(2,3)
a = pp.so3(n)
a, x.shape, x.gshape, x.gview(2), y

(so3Type Group:
 tensor([[ 0.2190, -0.7519,  0.1777],
         [ 0.3727,  0.3371,  0.6431]]),
 torch.Size([2, 1, 7]),
 torch.Size([2, 1]),
 SE3Type Group:
 tensor([[0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 1.]]),
 se3Type Group:
 tensor([[[ 1.0556,  0.4348,  0.0487, -0.3620, -0.6900,  0.6192],
          [-0.5872, -0.9864, -0.5331,  0.1646,  1.5045,  0.8371]],
 
         [[ 0.4859,  0.8837, -0.7357, -1.4727,  1.0960,  0.0509],
          [ 0.3445,  0.1424, -0.1667,  0.1331,  1.5327, -0.6572]]]))

### All arguments in PyTorch are supported

In [3]:
a = pp.randn_SO3(3, device="cuda:0", dtype=torch.double, requires_grad=True)
b = pp.identity_like(a, device="cpu")
a, b

(SO3Type Group:
 tensor([[ 0.1838, -0.3487,  0.0065,  0.9190],
         [-0.0114, -0.5920, -0.6765,  0.4380],
         [-0.5413,  0.6844, -0.4458,  0.1995]], device='cuda:0',
        dtype=torch.float64, requires_grad=True),
 SO3Type Group:
 tensor([[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]]))

# 2. Basic Operations

In [4]:
(x * y.Exp()).Log()

se3Type Group:
tensor([[[ 1.0556,  0.4348,  0.0487, -0.3620, -0.6900,  0.6192],
         [-0.5872, -0.9864, -0.5331,  0.1646,  1.5045,  0.8371]],

        [[ 0.4859,  0.8837, -0.7357, -1.4727,  1.0960,  0.0509],
         [ 0.3445,  0.1424, -0.1667,  0.1331,  1.5327, -0.6572]]])

# 3. Adjoint Transforms

In [5]:
X = pp.randn_Sim3(6, dtype=torch.double)
a = pp.randn_sim3(6, dtype=torch.double)
b = X.AdjT(a)
print((X * b.Exp() - a.Exp() * X).abs().mean() < 1e-7)

tensor(True)


In [6]:
X = pp.randn_SE3(8)
a = pp.randn_se3(8)
b = X.Adj(a)
print((b.Exp() * X - X * a.Exp()).abs().mean() < 1e-7)

tensor(True)


# 4. Grdients

In [7]:
x = pp.randn_so3(3, sigma=0.1, requires_grad=True, device="cuda")
assert x.is_leaf
loss = (x.Exp().Log()**2).sin().sum() # Just test, No phisical meaning
loss.backward()
loss, x.grad

(tensor(0.0651, device='cuda:0', grad_fn=<AliasBackward0>),
 tensor([[ 0.2175, -0.0455,  0.1925],
         [-0.1824,  0.2808,  0.2126],
         [-0.0128, -0.1233, -0.0324]], device='cuda:0'))

# 5. Test a Module

In [8]:
from torch import nn

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class TestNet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.weight = pp.Parameter(pp.randn_so3(n))

    def forward(self, x):
        return self.weight.Exp() * x


n,epoch = 4, 5
net = TestNet(n).cuda()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.2, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,4], gamma=0.5)

print(net.weight)
for i in range(epoch):
    optimizer.zero_grad()
    inputs = pp.randn_SO3(n).cuda()
    outputs = net(inputs)
    loss = outputs.abs().sum()
    loss.backward()
    optimizer.step()
    scheduler.step()
    print(loss)

print("Parameter:", count_parameters(net))
print(net.weight)

so3Type Group:
Parameter containing:
tensor([[-0.7629, -0.4715,  1.4965],
        [ 0.8281, -0.1802,  1.7058],
        [ 0.6121,  1.7299,  0.8937],
        [-1.5227,  1.2603,  1.5654]], device='cuda:0', requires_grad=True)
tensor(7.0401, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(7.1958, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(7.2081, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(6.3249, device='cuda:0', grad_fn=<AliasBackward0>)
tensor(7.2525, device='cuda:0', grad_fn=<AliasBackward0>)
Parameter: 12
so3Type Group:
Parameter containing:
tensor([[-0.6462, -0.3148,  0.3560],
        [-0.2815,  0.1488,  0.6884],
        [ 0.4615,  0.6315,  1.0637],
        [ 0.0093,  0.5062,  0.9129]], device='cuda:0', requires_grad=True)
